In [178]:
__author__ = "Philipp Kats"
__date__ = "2016_02_01"


import pandas as pd
import numpy as np
import datetime
import dateutil.parser
import time
import os

import sqlite3

In [241]:
def adapt_datetime(ts):
    return time.mktime(ts.timetuple())

sqlite3.register_adapter(datetime.datetime, adapt_datetime)

In [180]:
PWD = os.getenv('PWD')

path = '/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/gnip/enviroment4gnip/job1/PTDataDownload/downloads'

In [181]:
def getFiles(folder, format, full=True):
    for file in os.listdir(folder):
        if file.endswith(format):
            if full:
                yield os.path.join(folder, file)
            else:
                yield file

In [182]:
files = getFiles(path, '.json.gz', full=True)

In [183]:
import gzip
import json

def parseJsonArrayFile(path):
    with gzip.open(path, 'rb') as f:
        lines = f.read().split('\r')
        records = []
        for line in lines:
            if len(line.strip())>0:
                try:
                    records.append( json.loads(line.strip()) )
                except Exception, e:
                    pass
        return records


def parseAll(paths):
    records = []
    for path in paths:
        records.extend(parseJsonArrayFile(path))
    
    return records

In [184]:
l = parseJsonArrayFile('/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/gnip/enviroment4gnip/job1/PTDataDownload/downloads/20151116-20151116_580pe24mwz_2015_11_16_00_10_activities.json.gz')

In [185]:
records = parseAll(files)

In [186]:
len(records)

21259

In [187]:
### MY_SCHEMA
'''
id
timestamp (time)
geostamp (tuple of coordinates)
text
author uid
retweets
favorites
author_data
source
'''

'\nid\ntimestamp (time)\ngeostamp (tuple of coordinates)\ntext\nauthor uid\nretweets\nfavorites\nauthor_data\nsource\n'

In [264]:
def parseTweet(tweet, mode='dict'):
    if 'geo' in tweet:
        if mode=='dict':
            return {'id':int(tweet['id'].split(':')[-1]), 
                 'timestamp': int(time.mktime(ts.timetuple(dateutil.parser.parse(tweet['postedTime'])))),
                 'lon': tweet['geo']['coordinates'][0],
                 'lat': tweet['geo']['coordinates'][1],
                 'tweet':tweet['body'],
                 'user_id':int(tweet['actor']['id'].split(':')[-1]),
                 'rtwts':tweet['retweetCount'],
                 'fvrts':tweet['favoritesCount'],
                'source': tweet['generator']['displayName']}
        
        elif mode=='tpl':
            return (int(tweet['id'].split(':')[-1]),
                    int(time.mktime(dateutil.parser.parse(tweet['postedTime']).timetuple())),
                    tweet['geo']['coordinates'][0],
                    tweet['geo']['coordinates'][1],
                    tweet['body'],
                    int(tweet['actor']['id'].split(':')[-1]),
                    tweet['retweetCount'],
                    tweet['favoritesCount'],
                    tweet['generator']['displayName'])
    else:
        pass
#         print tweet.keys()



In [279]:
parcedRecords = [parseTweet(record, mode='tpl') for record in records if parseTweet(record, mode='tpl')!=None]

In [280]:
len(parcedRecords)

13029

In [282]:
parcedRecords[-1]

(666404912830500865,
 1447736287,
 40.71098913,
 -74.00404355,
 u'#NYCmissed Hot latin guy on R train every morning - m4m (Grand Ave.) https://t.co/tsKMh0DBhR',
 3073225894,
 0,
 0,
 u'dlvr.it')

In [283]:
tweetE = '''CREATE TABLE IF NOT EXISTS tweets (id INTEGER PRIMARY KEY, 
                                               timestamp INTEGER, 
                                               lon REAL, lat REAL, 
                                               tweet TEXT, 
                                               user_id INTEGER, 
                                               rtwts INTEGER,
                                               fvrts INTEGER,
                                               source TEXT)'''

In [284]:
# usersE = '''CREATE TABLE users (id INTEGER, 
#                                 timestamp INTEGER, raw TEXT, CONSTRAINT pk PRIMARY KEY (id, timestamp))'''

In [285]:
def setup(tweets):
    
    conn = sqlite3.connect(PWD + '/gnip_twitter.db')
    c = conn.cursor()
    c.execute(tweetE)
    c.executemany('INSERT OR IGNORE INTO tweets VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', tweets )
    conn.commit()
    conn.close()



In [286]:
setup(parcedRecords)

In [287]:
PWD

'/Users/casy/Dropbox/CUSP/GRA/QQ_GRA/code'